# Cvičenie 11: Zachráňme svet!

Asi najväčšia výhoda dobrých simulácií je to, že umožňujú urobiť rozhodnotia uvažovaním o možných dôsledkoch jednotlivých rozhodnutí a vybraním toho najvýhodnejšieho scenára. Na dnešnom cvičení si ukážeme príklad takejto pomocnej simulácie a to na príklade, ktorý všetci poznáme: účinok protiepidemiologických opatrení.

[Stiahnite si predpripravený projekt riešenia z tohto odkazu](sources/lab11/lab11.zip).

## Štruktúra simulácie

V našej simulácii budeme používať štyri moduly:

* `virus` - predstavuje vírus, ktorý sa šíri v populácii ľudí. Trieda je úplne pripravená, uchováva v sebe všetky potrebné informácie o víruse ohľadom jeho nákazlivosti a inkubačnej doby.
* `person` - člen populácie, ktorý môže byť nakazený vírusom. Trieda definuje spôsob infekcie, a uchováva v sebe informácie o aktuálnom zdravotnom stave človeka.
* `population` - uzavretá populácia, v ktorej sa šíri vírus. Trieda definuje spôsob šírenia nákazy, a zjednotí prácu a pohľad na ďalšie komponenty simulácie.
* `simulation` - hlavný súbor, ktorý definuje parametre simulácie a vizualizuje jej výsledky.

## `Virus`

Táto trieda definuje triedneho nepriateľa, samozrejme v jednodušenej forme. Ako môžete vidieť na konštruktore, vírus je popísaný štyrmi hodnotami:

* `incubation` - inkubačná doba - prvých niekoľko dní od nakazenia, v ktorých pacient ešte neprodukuje žiadne príznaky a pre naše účely ho budeme považovať za nenákazlivého. Teda aj keď je už nakazený, kým vírus je vo fáze inkubácie, človek nikoho nemôže nakaziť.
* `illness_length` - počet dní, po ktorých sa pacient uzdraví a získa imunitu voči vírusu.
* `max_transmission_rate` - maximálna miera infekčnosti počas choroby. Má hodnotu medzi 0 až 1, a vyjadruje pravdepodobnosť toho, že chorý človek nakazí niekoho v čase najvyššej infekčnosti.
* `immunity` - počet dní, kedy už uzdravený pacient má prirodzenú imunitu od nákazy a nemôže sa znova nakaziť.

Trieda definuje niekoľko getter metód pre jednotlivé členské premenné a metódu `get_transmission`, ktorá vráti pravdepodobnosť nakazenia v istý deň choroby. Pravdepodobnosť má [normálne rozdelenie](https://sk.wikipedia.org/wiki/Normálne_rozdelenie), pričom od dní 0 až `incubation` má hodnotu 0, a takisto od dňa `illness_length - incubation`. Maximálnu hodnotu `max_transmission_rate` dosiahne v polke choroby. Metóda je už implementovaná, môžete vizualizovať jej funkcionalitu pomocou hlavnej metódy.

![Zmena infekčnosti pčoas choroby](sources/lab11/change.png)

In [ ]:
import math


class Virus:
    def __init__(self, incubation, illness_length, max_transmission_rate, immunity):
        self.incubation = incubation
        self.illness_length = illness_length
        self.max_transmission_rate = max_transmission_rate
        self.immunity = immunity

    def get_immunity(self):
        return self.immunity

    def get_illness_length(self):
        return self.illness_length

    def get_incubation(self):
        return self.incubation

    def get_max_transmission_rate(self):
        return self.max_transmission_rate

    def get_gaussian_value(self, time_passed):
        smallest = self.incubation
        largest = self.illness_length - self.incubation
        std = (largest - smallest) / 4

        value = ((1 / (std * math.sqrt(math.tau))) * math.e ** (-0.5 * ((time_passed - self.illness_length // 2) / std) ** 2))

        return value

    def get_transmission(self, days_since_infected):
        if days_since_infected < self.incubation:
            return 0.0

        if days_since_infected > self.illness_length - self.incubation:
            return 0.0

        val = self.get_gaussian_value(days_since_infected)
        max_val = self.get_gaussian_value(self.illness_length // 2)
        norm_constant = self.max_transmission_rate / max_val

        return val * norm_constant


import matplotlib.pyplot as plt

virus = Virus(5, 14, 0.9, 90)
X = [i for i in range(1, 15)]
y = [virus.get_transmission(x) for x in X]

plt.plot(X, y)
plt.xlabel("days since infected")
plt.ylabel("probability of infecting")
plt.title("Change of transmission rate wrt days since infected")
plt.show()

## `Person`

Ďalšou časťou simulácie je človek (trieda `Person` v `person.py`). V konštruktore tejto triedy nastavíme členské premenné a to nasledovne:

* `infected = False` - vyjadruje, či človek je práve nakazený;
* `got_infected = None` - vyjadruje deň, kedy sa človek nakazil (pre práve aktuálnu chorobu);
* `has_virus = None` - smerník na objekt vírusu, ktorým sa človek nakazil;
* `got_cured = None` - deň, v ktorý sa človek naposledy uzdravil;
* `quarantined = False` - vyjadruje, či človek je práve v karanténe, alebo nie.

Getter metódy boli vytvorené pre členské premenné `quarantined` a `infected`. Trieda obsahuje tri ďalšie metódy, ktoré potrebujete implementovať.

**Úloha:** Implementujte metódu `infect`, ktorá reprezentuje jav nakazenia sa, pričom má tri parametre: `day` vyjadruje deň, v ktorý sa človek nakazí; `virus` je objekt vírusu, ktorým sa nakazí; a `quarantine` je `boolean` hodnota vyjadrujúca, či človeka pošleme do karantény alebo nie. Ak človek je už nakazený, metóda nič nerobí, a podobne sa nič neudeje ak človek má prirodzenú imunitu voči vírusu (ešte neprešlo viac dní od času uzdravenia sa ako dĺžka imunity). V ostatných prípadoch sa človek nakazí, čo reprezentujete nastavením potrebných členský premenných.

**Úloha:** Implementujte metódu `update`, ktorá aktualizuje zdravotný stav človeka pre deň `day`, a to nasledovne:

* ak človek nie je nakazený, nič neurobte;
* ak od dňa nakazenia prešlo viac dní ako dĺžka choroby, pacient sa uzdraví, čo reprezentujete vhodným nastavením relevantných členských premenných.

**Úloha:** Implementujte metódu `get_infectiousness`, ktorá vypočíta a vráti infekčnosť (pravdepodobnosť nakazenia ďalšieho človeka) pre daný deň `day`. Metóda vracia `0.0` ak človek nie je nakazený, v opačnom prípade vráti infekčnosť vírusu na základe počtu dní, ktoré prešli od času nakazenia.

In [ ]:
class Person:
    def __init__(self):
        self.infected = False
        self.got_infected = None
        self.has_virus = None
        self.got_cured = None
        self.quarantined = False

    def is_in_quarantine(self):
        return self.quarantined

    def is_infected(self):
        return self.infected

    def infect(self, day, virus, quarantine):
        # TODO: simulates a person getting infected
        # - if person already infected, do nothing
        # - if person immune after previous infection, do nothing
        # - otherwise infect person, set all relevant object attributes
        pass

    def update(self, day):
        # TODO: updates the person's state, potentially gets cured
        # - if person not infected, do nothing
        # - if person has been sick longer than the virus's illness duration,
        #   the person gets cured: set all relevant object attributes
        pass

    def get_infectiousness(self, day):
        # TODO: returns the probability of a person infecting someone else
        # - if the person is not infected, return 0.0
        # - otherwise return probability based on sickness duration
        return 0.0

## `Population`

Posledným prvkom simulácie je populácia ľudí (trieda `Population` v `population.py`), ktorú budeme považovať za uzavretú, t.j. ľudia neopúšťajú populáciu, a nikto nepríde do populácie počas simulácie (môžete to vnímať ako mesto v úplnom lockdowne). Populácia je popísaná hodnotami:

* `size` - veľkosť populácie, teda počet ľudí;
* `meets_per_day` - počet ľudí, ktorých ktorýkoľvek člen populácie stretne v jeden deň;
* `persons` - zoznam členov populácie (všetky objekty triedy `Person`);
* `quarantine` - vyjadruje stav použitia karantény (`True` alebo `False`). V našej simulácii bude karanténa jediný prostriedok tejto populácie v boji proti vírusu.

V triede nájdete dve už implementované metódy:

* `set_quarantine` - nastaví stav použitia karantény, slúži na aktualizáciu tejto hodnoty počas simulácie;
* `simulate_interaction` - simuluje stretnutie dvoch členov populácie, počas ktorého obaja môžu nakaziť druhého na základe infekčnosti (samozrejme len ak sú nakazení).

**Úloha:** Implementujte metódu `introduce_virus`, ktorá náhodne nakazí `patient_count` (počet) pacientov vírusom `virus` v deň `day`. V metóde nakazte vírusom ľudí z náhodne vybranej skupiny s veľkosťou `patient_count`.

**Úloha:** Implementujte metódu `get_moving_people`, ktorá vráti zoznam ľudí z populácie, ktorí sa budú pohybovať a stretávať ostatných ľudí. Pohybovať sa môže iba človek, ktorý práve nie je v karanténe. Ako parameter dostanete `mobility`, teda pomer tých ľudí, ktorí sú v pohybe, napr. pri hodnote `0.5` polovica populácie stretáva ostatných (môžu stretnúť aj takých ľudí, ktorí sú doma).

**Úloha:** Implementujte metódu `get_people_met`, ktorá vráti zoznam ľudí, ktorých človek `person` stretne v daný deň. Stretnúť môže hocikoho (bez ohľadu na to, či druhý človek bol vybraný ako pohybujúci sa človek alebo nie), ak ten druhý nie je v karanténe. Človek samozrejme nemôže stretnúť seba samého. Metóda vráti zoznam ľudí, ktorých človek stretne (ich počet je daný členskou premennou `meets_per_day`).

**Úloha:** Implementujte metódu `run_day`, ktorá nasimuluje priebeh jedného dňa, pričom každý simulovaný deň sa skladá z dvoch častí:

1. aktualizujte zdravotný stav každého člena populácie (aby sa ľudia uzdravili);
2. nasimulujte pohyb ľudí, pričom náhodne vyberiete pohybujúcich sa ľudí, a pre každého nasimulujete všetky stretnutia s ostatnými členmi populácie.

Metóda má dva parametre: `day` je aktuálny deň simulácie, a `mobility` je pomer ľudí v populácii, ktorí sa pohybujú.

**Úloha:** Implementujte metódu `get_number_of_infected`, ktorá vráti počet práve nakazených ľudí v populácii.

In [ ]:
import random


class Population:
    def __init__(self, size, meets_per_day, uses_quarantine):
        self.size = size
        self.meets_per_day = meets_per_day
        self.persons = list()
        for _ in range(self.size):
            self.persons.append(Person())

        self.quarantine = uses_quarantine

    def set_quarantine(self, quarantine):
        self.quarantine = quarantine

    def introduce_virus(self, day, virus, patient_count):
        # TODO: introduces a new virus to the population
        # infect a random sample of the population with the virus
        # sample size is given by patient_count
        pass

    def get_moving_people(self, mobility):
        # TODO: generates a random sample of people moving (meeting others)
        # only people not in quarantine can move
        # mobility is a number between 0 and 1
        # representing the percentage of people moving within the population
        return []

    def get_people_met(self, person):
        # TODO: generate a list of people a person meets
        # person can only meet people not in quarantine
        # returns the list of people
        return []

    def simulate_ineraction(self, person1, person2, day):
        # person 1 can infect person 2
        if random.random() < person1.get_infectiousness(day):
            person2.infect(day, self.virus, self.quarantine)

        # person 2 can infect person 1
        if random.random() < person2.get_infectiousness(day):
            person1.infect(day, self.virus, self.quarantine)

    def run_day(self, day, mobility):
        # TODO: simulates a day
        # 1. update people's state
        # 2. simulate people moving
        #   2.1. get list of people moving
        #   2.2. simulate all interactions
        pass

    def get_number_of_infected(self):
        # TODO: return the number of infected people within the population
        return 0

## Simulácia

Ostáva nám implementovať už len samotnú simuláciu v `simulation.py`. V `main` funkcii už máte pripravené volanie samotnej simulácie, chýba ešte ale funkcia `run_simulation`.

**Úloha:** Implementujte funkciu `run_simulation`, ktorá v sebe zahŕňa celú simuláciu šírenia vírusu `virus` v populácii `population` po `length` dní pri miere mobility `mobility` a s počtom prvých pacientov `initial_patient_count`. Vo funkcii urobte nasledovné kroky:

1. pridajte vírus do populácie (podľa parametrov funkcie);
2. iteratívne pre trvanie simulácie:
    * nasimulujte deň;
    * zistite počet nakazených a pridajte do zoznamu `counts`;
    * zaveďte dynamickú karanténu, t.j. karanténu zavediete ak počet nakazených je viac, ako 400, a karanténu zrušíte ak počet nakazených je menej ako 200.

Funkcia vracia zoznam `counts` s počtom nakazených v jednotlivých epizódach.

In [ ]:
def run_simulation(virus, initial_patient_count,
                   population, mobility,
                   length):
    # TODO: runs a simulation based on the parameters
    # 1. introduce virus to the population
    # 2. for the duration of the simulation:
    #   2.1. simulate day's events
    #   2.2. get number of infected and append it to counts
    #   2.3. introduce dynamic quarantine:
    #          - use quarantine if more than 400 people are infected
    #          - stop using quarantine if less than 200 people are infected
    counts = list()

    return counts


covid = Virus(incubation=5, illness_length=14, max_transmission_rate=0.8, immunity=90)
population = Population(size=10000, meets_per_day=6, uses_quarantine=False)

points = run_simulation(covid, 6, population, 0.1, 500)

plt.plot(points)
plt.show()

## Možné rozšírenia

Ak chcete získať ďalšie skúsenosti so simuláciami, Vaše riešenie môžete rozšíriť napríklad nasledovne:

1. Zmeňte mobilitu počas simulácie podobne ako použitie karantény.
2. Pošlite pacienta do karantény len po uplynutí inkubačnej doby.
3. Pridajte podporu pre šírenie viacerých vírusov v populácii.
4. Simuláciu upravte tak, aby ľudia dodržiavali karanténu len s istou pravdepodobnosťou.
5. Zaveďte koncept testovania do simulácie, t.j. chorých pošlite do karantény ešte počas inkubačnej doby (bod 2) ak boli pozitívne testovaní.
6. Pridajte presnosť testovania, aby test nie vždy dával správny výsledok.
7. Pridajte do simulácie viaceré testy s rôznou presnosťou.